# CatBoost с текстовыми фичами
CatBoost неплохо работает с текстовыми фичами. Можно использовать эту модель как бейзлайн.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from datasets import load_dataset
from catboost import CatBoostClassifier, Pool
from tqdm import tqdm
import pickle
import warnings

warnings.filterwarnings('ignore')
tqdm.pandas()

In [3]:
# загрузка датасета
df = load_dataset("MonoHime/ru_sentiment_dataset")
df_train, df_test = df['train'].to_pandas(), df['validation'].to_pandas()

In [4]:
df_train, df_test = df_train.drop(['Unnamed: 0'], axis=1), df_test.drop(['Unnamed: 0'], axis=1)

In [5]:
df_train.shape, df_test.shape

((189891, 2), (21098, 2))

In [6]:
df_train.head()

,text,sentiment
0,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,Непорядочное отношение к своим работникам Рабо...,2
3,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,У меня машина в руках 5 лет и это п...,1


In [7]:
df_train, df_val = train_test_split(df_train, test_size=0.2, stratify=df_train['sentiment'], random_state=42)

In [8]:
df_train.shape, df_val.shape, df_test.shape

((151912, 2), (37979, 2), (21098, 2))

## Эвристические признаки

In [9]:
import pymorphy2
import re
from functools import lru_cache

morph = pymorphy2.MorphAnalyzer()

In [10]:
stop_words = [
    # Местоимения (личные, притяжательные, указательные — без контекста бесполезны)
    "я", "ты", "он", "она", "оно", "мы", "вы", "они",
    "меня", "тебя", "его", "её", "нас", "вас", "их",
    "мне", "тебе", "ему", "ей", "нам", "вам", "им",
    "мной", "тобой", "им", "ней", "нами", "вами", "ими",
    "мой", "твой", "его", "её", "наш", "ваш", "их",
    "этот", "эта", "это", "эти",
    "тот", "та", "то", "те",
    "весь", "вся", "всё", "все",
    "сам", "сама", "само", "сами",
    "кто", "что", "какой", "какая", "какое", "какие",
    "чей", "чья", "чьё", "чьи",
    "который", "которая", "которое", "которые",

    # Предлоги (чисто грамматические)
    "в", "во", "на", "с", "со", "из", "от", "до", "у", "о", "об", "по", "за", "над", "под",
    "к", "ко", "перед", "через", "без", "для", "про", "при", "между", "около", "вокруг",
    "сквозь", "после", "против", "вдоль", "вблизи", "внутри", "снаружи", "напротив",
    "вслед", "вместо", "вне", "внутрь", "вверх", "вниз", "вперёд", "назад",

    # Союзы
    "и", "а", "но", "да", "или", "либо", "зато", "что", "чтобы", "как", "если", "когда",
    "пока", "потому", "хотя", "несмотря", "ли", "тоже", "также", "ни", "едва",

    # Частицы (чисто грамматические, не усиливающие/не ослабляющие)
    "не", "ни", "бы", "пусть", "давай", "давайте", "то", "либо", "нибудь", "ка",

    # Вспомогательные глаголы и связки
    "быть", "есть", "являться", "становиться", "делать", "сделать", "иметь", "мочь",
    "хотеть", "надо", "нужно", "следует", "стоит", "бывает"
]

In [11]:
positive_words = [
    # Эмоции и чувства
    "радость", "счастье", "восторг", "блаженство", "наслаждение", "удовольствие",
    "ликование", "восхищение", "вдохновение", "энтузиазм", "воодушевление",
    "улыбка", "смех", "веселье", "бодрость", "оптимизм", "надежда", "вера",
    "доверие", "любовь", "нежность", "забота", "доброта", "щедрость", "милосердие",
    "благодарность", "признательность", "уважение", "восхищение", "гордость",
    "успех", "триумф", "победа", "гармония", "спокойствие", "мир", "согласие",
    "свобода", "независимость", "достоинство", "честь", "благородство",

    # Положительные качества личности
    "добрый", "хороший", "честный", "отзывчивый", "внимательный", "заботливый",
    "преданный", "верный", "надёжный", "щедрый", "милосердный", "благородный",
    "вежливый", "тактичный", "скромный", "трудолюбивый", "настойчивый",
    "целеустремлённый", "ответственный", "умный", "талантливый", "креативный",
    "изобретательный", "инициативный", "энергичный", "активный", "бодрый",
    "здоровый", "сильный", "выносливый", "гибкий", "уравновешенный", "спокойный",
    "терпеливый", "дружелюбный", "открытый", "искренний", "прямолинейный",
    "чуткий", "эмпатичный", "гуманный", "гостеприимный", "обаятельный",
    "харизматичный", "обходительный", "предусмотрительный", "благоразумный",

    # Действия и глаголы
    "радоваться", "улыбаться", "смеяться", "веселиться", "наслаждаться",
    "восхищаться", "вдохновлять", "вдохновляться", "помогать", "поддерживать",
    "заботиться", "дарить", "делиться", "благодарить", "хвалить", "одобрять",
    "прощать", "обнимать", "любить", "уважать", "ценить", "доверять", "верить",
    "побеждать", "успевать", "преуспевать", "развиваться", "расти", "учиться",
    "творить", "создавать", "строить", "улучшать", "исправлять", "восстанавливать",
    "оживать", "ожидать", "надеяться", "мечтать", "стремиться", "достигать",
    "получать", "находить", "открывать", "радовать", "восстанавливать", "воскрешать",

    # Состояния и абстрактные понятия
    "благо", "добро", "красота", "истина", "справедливость", "гармония",
    "порядок", "ясность", "чистота", "свежесть", "лёгкость", "простота",
    "ясность", "понимание", "согласие", "единство", "согласованность", "созвучие",
    "процветание", "благоухание", "сияние", "свет", "тепло", "уют", "комфорт",
    "стабильность", "надёжность", "безопасность", "защищённость", "достаток",
    "изобилие", "плодородие", "рост", "расцвет", "возрождение", "оживление",

    # Наречия и усилители
    "прекрасно", "замечательно", "отлично", "чудесно", "восхитительно",
    "великолепно", "потрясающе", "фантастически", "безупречно", "идеально",
    "успешно", "гладко", "легко", "свободно", "радостно", "весело", "игриво",
    "нежно", "тепло", "искренне", "честно", "открыто", "щедро", "благородно",
    "мудро", "умело", "ловко", "мастерски", "профессионально", "уверенно",
    "спокойно", "уравновешенно", "терпеливо", "внимательно", "заботливо",

    # Дополнительные позитивные слова
    "солнце", "цветок", "весна", "рассвет", "звезда", "небо", "море", "луг",
    "праздник", "подарок", "сюрприз", "чудо", "волшебство", "сказка", "мечта",
    "план", "цель", "намерение", "желание", "стремление", "воля", "сила",
    "здоровье", "жизнь", "любимый", "дорогой", "близкий", "родной", "друг",
    "союз", "партнёрство", "сотрудничество", "поддержка", "помощь", "семья",
    "дом", "очаг", "мир", "тишина", "покой", "баланс", "равновесие", "ясность",
    "просветление", "озарение", "прозрение", "вдохновение", "муза", "талант",
    "гений", "дар", "способность", "возможность", "шанс", "удача", "везение",
    "благоприятный", "успешный", "плодотворный", "эффективный", "полезный",
    "ценный", "важный", "значимый", "нужный", "желанный", "ожидаемый", "радостный",

    # Общая оценка
    "отлично", "классно", "супер", "круто", "огонь", "топ", "бомба", "вау", "ух ты",
    "восхитительно", "замечательно", "прекрасно", "идеально", "безупречно", "фантастика",
    "лучше", "лучший", "надёжный", "качественный", "стильный", "современный", "удобный",

    # Эмоции и реакции
    "рад", "доволен", "в восторге", "восхищён", "впечатлён", "обрадован", "счастлив",
    "улыбается", "смеётся", "весело", "уютно", "тепло", "приятно", "комфортно", "легко",

    # Оценка сервиса/обслуживания
    "вежливый", "внимательный", "помогли", "поддержали", "быстро", "оперативно",
    "вовремя", "профессионально", "грамотно", "чётко", "ясно", "понятно", "дружелюбно",
    "приветливо", "обходительно", "заботливо", "ответили", "решили", "исправили",

    # Оценка товара/продукта
    "работает", "функционирует", "заряжается", "держит", "не глючит", "не тормозит",
    "яркий", "насыщенный", "вкусный", "ароматный", "сочный", "нежный", "мягкий", "лёгкий",
    "долговечный", "прочный", "надёжный", "эргономичный", "эстетичный", "красивый",

    # Доверие и рекомендации
    "доверяю", "рекомендую", "советую", "берите", "покупайте", "закажите", "попробуйте",
    "стоит", "выгодно", "дёшево", "недорого",

    # Удовлетворённость
    "подходит", "удобно", "практично", "полезно", "нужно", "важно", "ценю", "благодарю"
]

In [12]:
negative_words = [
    # Эмоции и состояния
    "грусть", "печаль", "тоска", "уныние", "отчаяние", "безысходность", "апатия",
    "депрессия", "страх", "ужас", "паника", "тревога", "волнение", "беспокойство",
    "раздражение", "злость", "ярость", "гнев", "ненависть", "злоба", "озлобленность",
    "обида", "разочарование", "неудовольствие", "недовольство", "досада", "досада",
    "стыд", "виноватость", "раскаяние", "сожаление", "зависть", "ревность",
    "подозрение", "недоверие", "цинизм", "пессимизм", "безверие", "отчуждение",
    "одиночество", "изоляция", "потеря", "горе", "беда", "несчастье", "бедствие",
    "катастрофа", "провал", "поражение", "неудача", "крах", "банкротство",

    # Негативные качества личности
    "злой", "жестокий", "бессердечный", "бездушный", "эгоистичный", "корыстный",
    "жадный", "алчный", "лживый", "обманщик", "коварный", "вероломный",
    "подлый", "низкий", "гадкий", "мерзкий", "грязный", "нечестный", "надменный",
    "высокомерный", "презрительный", "грубый", "хамский", "агрессивный",
    "вспыльчивый", "раздражительный", "нудный", "скучный", "ленивый", "бездельник",
    "трусливый", "робкий", "неуверенный", "замкнутый", "подозрительный",
    "мстительный", "злопамятный", "интриган", "лицемер", "фальшивый", "надутый",
    "вредный", "заносчивый", "самодовольный", "недоброжелательный", "враждебный",

    # Действия и глаголы
    "плакать", "страдать", "мучиться", "болеть", "ранить", "обижать", "оскорблять",
    "унижать", "презирать", "ненавидеть", "завидовать", "ревновать", "обманывать",
    "врать", "скрывать", "прятать", "подозревать", "бояться", "дрожать", "кричать",
    "ругаться", "ссориться", "воевать", "нападать", "уничтожать", "ломать", "портить",
    "губить", "вредить", "мешать", "мешать", "препятствовать", "саботировать",
    "отказывать", "лишать", "терять", "проваливать", "проигрывать", "падать",
    "разрушать", "разваливаться", "гнить", "портиться", "исчезать", "умирать",
    "задыхаться", "стонать", "вопить", "жаловаться", "ныть", "ворчать", "проклинать",

    # Состояния и абстракции
    "боль", "страдание", "мука", "пытка", "угнетение", "давление", "насилие",
    "тирания", "деспотизм", "несправедливость", "обман", "ложь", "фальшь",
    "лицемерие", "предательство", "измена", "клевета", "сплетня", "интрига",
    "хаос", "беспорядок", "разруха", "грязь", "тлен", "смерть", "гибель", "конец",
    "тупик", "безысходность", "пустота", "бессмысленность", "абсурд", "безумие",
    "безумие", "помешательство", "паранойя", "ненависть", "вражда", "конфликт",
    "война", "раздор", "неприязнь", "враждебность", "недоброжелательность",

    # Наречия и усилители негатива
    "ужасно", "кошмарно", "отвратительно", "мерзко", "гадко", "противно",
    "неприятно", "тошнотворно", "безобразно", "скверно", "дурно", "плохо",
    "неудачно", "бесполезно", "бессмысленно", "тщетно", "напрасно", "впустую",
    "безрезультатно", "неправильно", "ошибочно", "фальшиво", "лицемерно",
    "жестоко", "бессердечно", "грубо", "хамски", "агрессивно", "враждебно",

    # Дополнительные негативные слова
    "тень", "тьма", "ночь", "холод", "лед", "смерть", "болезнь", "рана", "кровь",
    "слеза", "крик", "стон", "плач", "вопль", "ужас", "кошмар", "призрак", "чудовище",
    "дьявол", "ад", "проклятие", "порок", "грех", "падение", "разврат", "алкоголь",
    "наркотик", "зависимость", "разлад", "разрыв", "развод", "изгнание", "изоляция",
    "тюрьма", "цепи", "рабство", "угнетение", "эксплуатация", "бедность", "нищета",
    "голод", "холод", "болезнь", "инвалидность", "старость", "забвение", "заброшенность",
    "забвение", "забытость", "беспомощность", "беззащитность", "уязвимость",
    "опасность", "угроза", "риск", "авария", "крушение", "пожар", "наводнение",
    "землетрясение", "эпидемия", "чума", "голод", "война", "террор", "насилие",

    # Общая оценка
    "ужасно", "кошмарно", "отвратительно", "мерзко", "паршиво", "фу",
    "разочарование", "провал", "фейк", "обман", "пустышка",

    # Эмоции и реакции
    "разочарован", "недоволен", "злился", "разозлился", "обиделся", "расстроился",
    "раздражает", "бесит", "надоело", "устал", "усталость", "апатия",

    # Проблемы с сервисом/обслуживанием
    "игнорируют", "молчат", "кинули", "кидают", "обманули",
    "задержали", "потеряли", "сломали", "испортили", "грубят",
    "хамят", "некомпетентные", "тупят", "перекладывают", "отфутболили",

    # Проблемы с товаром/продуктом
    "сломался", "глючит", "тормозит", "виснет", "разрядился",
    "воняет", "горький", "пресный", "сухой", "жёсткий", "неудобный",
    "тяжёлый", "громоздкий", "хлипкий", "дешёвка", "подделка",
    "царапает", "давит", "жмёт",

    # Финансовые претензии
    "дорого", "переплатил", "завысили", "надули", "обсчитали",

    # Недоверие и предостережения
    "мошенники", "ловушка", "развод"
]

In [13]:
intensifiers = [
    # Стандартные усилители
    "очень", "сильно", "крайне", "чрезвычайно", "невероятно", "безумно", "адски",
    "страшно", "ужасно", "дико", "бешено", "чертовски", "просто", "абсолютно",
    "полностью", "совершенно", "вполне", "целиком", "настоятельно", "исключительно",

    # Разговорные и интернет-усилители
    "прям", "прямиком", "вообще", "полностью", "до безумия", "до ужаса", "до дна",
    "всё-таки", "таки", "ещё", "ещё бы", "намного", "гораздо", "значительно",
    "в разы", "в сто раз", "в тысячу раз", "максимально", "супер", "ультра", "гипер",

    # Усилители положительной/отрицательной оценки (в зависимости от контекста)
    "идеально", "безупречно", "абсолютно", "фактически", "действительно", "в самом деле"
]

In [14]:
diminishers = [
    # Смягчители и ограничители
    "немного", "чуть", "чуть-чуть", "слегка", "легко", "едва", "едва-едва",
    "почти", "приблизительно", "примерно", "вроде", "как бы", "типа", "наверное",
    "возможно", "похоже", "кажется", "предположительно", "отчасти", "частично",
    "в какой-то степени", "в некоторой степени", "в меру", "умеренно", "достаточно",

    # Слова неуверенности и сомнения
    "скорее", "вряд ли", "вряд", "едва ли", "всё же", "всё-таки", "таки",
    "лишь", "только", "всего", "всего лишь", "просто", "в общем", "в целом",

    # Уменьшительно-ласкательные (в разговорной речи могут ослаблять серьёзность)
    "малость", "маленько", "чуток"
]

In [15]:
with open('data/bad_words.pickle', 'rb') as file:
    bad_words = pickle.load(file)

In [ ]:
# функция для лемматизации

@lru_cache(maxsize=1_000_000)
def lemmatize_word(word):
    """Приводит одно слово к его нормальной форме (лемме)."""
    parsed_word = morph.parse(word)
    
    # самый вероятный вариант.
    if parsed_word:
        return parsed_word[0].normal_form
    return word

In [17]:
stop_words = set([lemmatize_word(word) for word in stop_words])
positive_words = set([lemmatize_word(word) for word in positive_words])
negative_words = set([lemmatize_word(word) for word in negative_words])
intensifiers = set([lemmatize_word(word) for word in intensifiers])
diminishers = set([lemmatize_word(word) for word in diminishers])
bad_words = set([lemmatize_word(word) for word in bad_words])

In [18]:
# функции для вычисления признаков
def text_length(text: str, *args) -> int:
    return len(text)

def avg_word_length(text: str, *args) -> float:
    return np.mean([len(x) for x in text.split()]).item()

def punct_marks_ratio(text: str, *args) -> float:
    punct_count = sum(text.count(mark) for mark in ',.;:!?\"\'')
    total_count = len(text)
    return punct_count / (total_count + 1e-10)

def exclamation_ratio(text: str, *args) -> float:
    exclamation_count = text.count('!')
    total_count = len(text)
    return exclamation_count / (total_count + 1e-10)

def question_ratio(text: str, *args) -> float:
    question_count = text.count('?')
    total_count = len(text)
    return question_count / (total_count + 1e-10)

def caps_words_ratio(text: str, cleaned_text: str, *args) -> float:
    words = cleaned_text.split()
    caps_words_count = sum(word.isupper() for word in words)
    words_total_count = len(words)
    return caps_words_count / (words_total_count + 1e-10)

def caps_symbols_ratio(text: str, cleaned_text: str, *args) -> float:
    caps_symbols_count = sum(x.isupper() for x in cleaned_text)
    total_count = len(cleaned_text)
    return caps_symbols_count / (total_count + 1e-10)

def unique_words_ratio(text: str, cleaned_text: str, lemmatized_text: str) -> float:
    unique_words_count = len(set(lemmatized_text.split()))
    total_words_count = len(lemmatized_text.split())
    return unique_words_count / (total_words_count + 1e-10)

def stop_words_ratio(text: str, cleaned_text: str, lemmatized_text: str) -> float:
    stop_words_count = sum(word in stop_words for word in lemmatized_text.split())
    total_words_count = len(lemmatized_text.split())
    return stop_words_count / (total_words_count + 1e-10)

def positive_words_ratio(text: str, cleaned_text: str, lemmatized_text: str) -> float:
    positive_words_count = sum(word in positive_words for word in lemmatized_text.split())
    total_words_count = len(lemmatized_text.split())
    return positive_words_count / (total_words_count + 1e-10)

def negative_words_ratio(text: str, cleaned_text: str, lemmatized_text: str) -> float:
    negative_words_count = sum(word in negative_words for word in lemmatized_text.split())
    total_words_count = len(lemmatized_text.split())
    return negative_words_count / (total_words_count + 1e-10)

def intensifiers_ratio(text: str, cleaned_text: str, lemmatized_text: str) -> float:
    intensifier_words_count = sum(word in intensifiers for word in lemmatized_text.split())
    total_words_count = len(lemmatized_text.split())
    return intensifier_words_count / (total_words_count + 1e-10)

def diminishers_ratio(text: str, cleaned_text: str, lemmatized_text: str) -> float:
    diminisher_words_count = sum(word in diminishers for word in lemmatized_text.split())
    total_words_count = len(lemmatized_text.split())
    return diminisher_words_count / (total_words_count + 1e-10)

def bad_words_ratio(text: str, cleaned_text: str, lemmatized_text: str) -> float:
    bad_words_count = sum(word in bad_words for word in lemmatized_text.split())
    total_words_count = len(lemmatized_text.split())
    return bad_words_count / (total_words_count + 1e-10)

In [19]:
def prepare_features(df: pd.DataFrame) -> pd.DataFrame:
    """Вычисляет все эвристические признаки для датасета"""
    
    df = df.copy()
    
    features_functions = [
        ('text_length', text_length),
        ('avg_word_length', avg_word_length),
        ('punct_marks_ratio', punct_marks_ratio),
        ('exclamation_ratio', exclamation_ratio),
        ('question_ratio', question_ratio),
        ('caps_words_ratio', caps_words_ratio),
        ('caps_symbols_ratio', caps_symbols_ratio),
        ('stop_words_ratio', stop_words_ratio),
        ('unique_words_ratio', unique_words_ratio),
        ('positive_words_ratio', positive_words_ratio),
        ('negative_words_ratio', negative_words_ratio),
        ('intensifiers_ratio', intensifiers_ratio),
        ('diminishers_ratio', diminishers_ratio),
        ('bad_words_ratio', bad_words_ratio),
    ]

    features = [x[0] for x in features_functions]
    functions = [x[1] for x in features_functions]

    df['cleaned_text'] = df['text'].progress_apply(lambda text: re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9]', ' ', text))
    df['lemmatized_text'] = df['cleaned_text'].progress_apply(lambda text: ' '.join([lemmatize_word(word) for word in text.split()]))
    df[features] = df.progress_apply(
        lambda row: [func(row.text, row.cleaned_text, row.lemmatized_text) for func in functions],
        axis=1, result_type='expand'
    )

    return df

In [20]:
df_train = prepare_features(df_train)

100%|██████████| 151912/151912 [00:51<00:00, 2953.99it/s]


In [21]:
df_val = prepare_features(df_val)

100%|██████████| 37979/37979 [00:12<00:00, 2970.64it/s]


In [22]:
df_test = prepare_features(df_test)

100%|██████████| 21098/21098 [00:07<00:00, 2930.56it/s]


In [23]:
df_train.shape, df_val.shape, df_test.shape

((151912, 18), (37979, 18), (21098, 18))

## Обучение моделей

### 1. Модель только на текстовых фичах

In [24]:
pool_train = Pool(
    data=df_train[['text']],
    label=df_train['sentiment'],
    text_features=['text'],
)

pool_val = Pool(
    data=df_val[['text']],
    label=df_val['sentiment'],
    text_features=['text'],
)

pool_test = Pool(
    data=df_test[['text']],
    label=df_test['sentiment'],
    text_features=['text'],
)

In [25]:
model = CatBoostClassifier(
    iterations=1000,
    max_depth=8,
    learning_rate=5e-2,
    eval_metric='Accuracy',
    early_stopping_rounds=50,
    use_best_model=True,
    random_seed=42,
    verbose=50,
)

model.fit(pool_train, eval_set=pool_val)

0:	learn: 0.6861077	test: 0.6945154	best: 0.6945154 (0)	total: 1.16s	remaining: 19m 19s
50:	learn: 0.7068039	test: 0.7097343	best: 0.7097343 (50)	total: 52s	remaining: 16m 8s
100:	learn: 0.7181789	test: 0.7198188	best: 0.7198188 (100)	total: 1m 48s	remaining: 16m 6s
150:	learn: 0.7248999	test: 0.7266121	best: 0.7268227 (149)	total: 2m 53s	remaining: 16m 14s
200:	learn: 0.7289878	test: 0.7292978	best: 0.7292978 (200)	total: 3m 54s	remaining: 15m 30s
250:	learn: 0.7342014	test: 0.7337213	best: 0.7338792 (247)	total: 4m 54s	remaining: 14m 37s
300:	learn: 0.7390924	test: 0.7373549	best: 0.7373549 (300)	total: 5m 50s	remaining: 13m 32s
350:	learn: 0.7434831	test: 0.7400405	best: 0.7403039 (348)	total: 6m 48s	remaining: 12m 34s
400:	learn: 0.7467350	test: 0.7419363	best: 0.7420680 (398)	total: 7m 44s	remaining: 11m 33s
450:	learn: 0.7497433	test: 0.7435951	best: 0.7435951 (450)	total: 8m 40s	remaining: 10m 33s
500:	learn: 0.7524685	test: 0.7447537	best: 0.7450433 (487)	total: 9m 36s	remainin

In [26]:
y_test = df_test['sentiment']
y_test_pred = model.predict(pool_test)

In [28]:
print('Accuracy: %.4f\nPrecision_macro: %.4f\nRecall_macro: %.4f' % (accuracy_score(y_test, y_test_pred),
                                                                     precision_score(y_test, y_test_pred, average='macro'),
                                                                     recall_score(y_test, y_test_pred, average='macro')))

Accuracy: 0.7378
Precision_macro: 0.7288
Recall_macro: 0.7359


In [31]:
# сохранение модели
model.save_model('artifacts/catboost/text_only_model.cbm')

### 2. Модель на всех признаках (текстовые + эвристические)

In [32]:
features = [
    'text', 'lemmatized_text',
    'text_length', 'avg_word_length', 'punct_marks_ratio',
    'exclamation_ratio', 'question_ratio', 'caps_words_ratio',
    'caps_symbols_ratio', 'stop_words_ratio', 'unique_words_ratio',
    'positive_words_ratio', 'negative_words_ratio', 'intensifiers_ratio',
    'diminishers_ratio', 'bad_words_ratio',
]

In [33]:
pool_train = Pool(
    data=df_train[features],
    label=df_train['sentiment'],
    text_features=['text', 'lemmatized_text'],
)

pool_val = Pool(
    data=df_val[features],  
    label=df_val['sentiment'],
    text_features=['text', 'lemmatized_text'],
)

pool_test = Pool(   
    data=df_test[features],
    label=df_test['sentiment'],
    text_features=['text', 'lemmatized_text'],
)

In [34]:
model = CatBoostClassifier(
    iterations=1000,
    max_depth=8,
    learning_rate=5e-2,
    eval_metric='Accuracy',
    early_stopping_rounds=50,
    use_best_model=True,
    random_seed=42,
    verbose=50,
)

model.fit(pool_train, eval_set=pool_val)

0:	learn: 0.6958239	test: 0.6987019	best: 0.6987019 (0)	total: 1.98s	remaining: 32m 54s
50:	learn: 0.7281913	test: 0.7280866	best: 0.7280866 (50)	total: 1m 38s	remaining: 30m 41s
100:	learn: 0.7412778	test: 0.7418573	best: 0.7418573 (100)	total: 3m 24s	remaining: 30m 23s
150:	learn: 0.7499934	test: 0.7497828	best: 0.7499934 (145)	total: 5m 14s	remaining: 29m 28s
200:	learn: 0.7563984	test: 0.7538376	best: 0.7538376 (200)	total: 7m 4s	remaining: 28m 6s
250:	learn: 0.7619477	test: 0.7582611	best: 0.7582875 (245)	total: 8m 54s	remaining: 26m 35s
300:	learn: 0.7675496	test: 0.7629216	best: 0.7632376 (295)	total: 10m 44s	remaining: 24m 56s
350:	learn: 0.7718218	test: 0.7656863	best: 0.7656863 (350)	total: 12m 32s	remaining: 23m 12s
400:	learn: 0.7755214	test: 0.7680824	best: 0.7682930 (399)	total: 14m 22s	remaining: 21m 27s
450:	learn: 0.7788851	test: 0.7700835	best: 0.7700835 (450)	total: 16m 11s	remaining: 19m 42s
500:	learn: 0.7818079	test: 0.7716106	best: 0.7719793 (496)	total: 18m 2s	r

In [35]:
y_test = df_test['sentiment']
y_test_pred = model.predict(pool_test)

In [36]:
print('Accuracy: %.4f\nPrecision_macro: %.4f\nRecall_macro: %.4f' % (accuracy_score(y_test, y_test_pred),
                                                                     precision_score(y_test, y_test_pred, average='macro'),
                                                                     recall_score(y_test, y_test_pred, average='macro')))

Accuracy: 0.7725
Precision_macro: 0.7634
Recall_macro: 0.7654


In [37]:
# сохранение модели
model.save_model('artifacts/catboost/text_and_heuristics_model.cbm')

## 3. Эмбеддинги BERT + эвристики

In [38]:
from transformers import AutoTokenizer, AutoModel
import torch
import math

In [39]:
# облегченная модель BERT для эмбеддингов

model_name = 'cointegrated/rubert-tiny2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

print(f"Модель загружена! Параметров: {model.num_parameters():,}")

Модель загружена! Параметров: 29,193,768


In [40]:
device = 'cpu' 
model = model.to(device)

In [41]:
@lru_cache(maxsize=1_000_000)
def get_embedding(text):
    """Получение эмбеддинга для одного текста"""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        # среднее по токенам (mean pooling)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    
    return embedding.cpu().numpy()

In [42]:
get_embedding('проверка эмбеддера').shape

(312,)

In [43]:
EMBEDDING_SIZE = 312

In [44]:
embedding_columns = [f'emb_{i}' for i in range(EMBEDDING_SIZE)]

df_train[embedding_columns] = df_train.progress_apply(lambda row: get_embedding(row['text']), axis=1, result_type='expand')
df_val[embedding_columns]   = df_val.progress_apply(lambda row: get_embedding(row['text']), axis=1, result_type='expand')
df_test[embedding_columns]  = df_test.progress_apply(lambda row: get_embedding(row['text']), axis=1, result_type='expand')

100%|██████████| 21098/21098 [01:38<00:00, 214.75it/s]


In [45]:
df_train.shape, df_val.shape, df_test.shape

((151912, 330), (37979, 330), (21098, 330))

Чтобы не обучаться на 312-мерных эмбеддингах, можно понизить размерность с помощью PCA до 64:

In [46]:
from sklearn.decomposition import PCA

In [47]:
pca_model = PCA(n_components=64)

pca_embeddings_train = pca_model.fit_transform(df_train[embedding_columns])
pca_embeddings_val = pca_model.transform(df_val[embedding_columns])
pca_embeddings_test = pca_model.transform(df_test[embedding_columns])

pca_features = [f'emb_pca_{i}' for i in range(pca_model.n_components_)]

df_train[pca_features] = pca_embeddings_train
df_val[pca_features] = pca_embeddings_val
df_test[pca_features] = pca_embeddings_test

In [48]:
df_train.shape, df_val.shape, df_test.shape

((151912, 394), (37979, 394), (21098, 394))

In [49]:
features = [
    'text_length', 'avg_word_length', 'punct_marks_ratio',
    'exclamation_ratio', 'question_ratio', 'caps_words_ratio',
    'caps_symbols_ratio', 'stop_words_ratio', 'unique_words_ratio',
    'positive_words_ratio', 'negative_words_ratio', 'intensifiers_ratio',
    'diminishers_ratio', 'bad_words_ratio',
] + pca_features

In [50]:
pool_train = Pool(
    data=df_train[features],
    label=df_train['sentiment']
)

pool_val = Pool(
    data=df_val[features],  
    label=df_val['sentiment'],
)

pool_test = Pool(   
    data=df_test[features],
    label=df_test['sentiment'],
)

In [51]:
model = CatBoostClassifier(
    iterations=1000,
    max_depth=8,
    learning_rate=5e-2,
    eval_metric='Accuracy',
    early_stopping_rounds=100,
    use_best_model=True,
    verbose=50,
    random_seed=42,
)

model.fit(pool_train, eval_set=pool_val)

0:	learn: 0.6224393	test: 0.6215277	best: 0.6215277 (0)	total: 63ms	remaining: 1m 2s
50:	learn: 0.6940071	test: 0.6913557	best: 0.6913557 (50)	total: 3.69s	remaining: 1m 8s
100:	learn: 0.7140384	test: 0.7082335	best: 0.7082335 (100)	total: 7.07s	remaining: 1m 2s
150:	learn: 0.7258149	test: 0.7177388	best: 0.7177388 (150)	total: 10.3s	remaining: 58.1s
200:	learn: 0.7346161	test: 0.7229522	best: 0.7231628 (198)	total: 13.9s	remaining: 55.3s
250:	learn: 0.7421139	test: 0.7268754	best: 0.7268754 (250)	total: 17.1s	remaining: 50.9s
300:	learn: 0.7488151	test: 0.7295874	best: 0.7295874 (300)	total: 21.2s	remaining: 49.2s
350:	learn: 0.7547067	test: 0.7329314	best: 0.7330630 (347)	total: 24.8s	remaining: 45.9s
400:	learn: 0.7603415	test: 0.7357224	best: 0.7359857 (398)	total: 28.2s	remaining: 42.1s
450:	learn: 0.7651403	test: 0.7383027	best: 0.7383027 (450)	total: 31.4s	remaining: 38.3s
500:	learn: 0.7696035	test: 0.7393823	best: 0.7394350 (496)	total: 34.5s	remaining: 34.4s
550:	learn: 0.774

In [52]:
y_test = df_test['sentiment']
y_test_pred = model.predict(pool_test)

In [53]:
print('Accuracy: %.4f\nPrecision_macro: %.4f\nRecall_macro: %.4f' % (accuracy_score(y_test, y_test_pred),
                                                                     precision_score(y_test, y_test_pred, average='macro'),
                                                                     recall_score(y_test, y_test_pred, average='macro')))

Accuracy: 0.7437
Precision_macro: 0.7342
Recall_macro: 0.7288


In [54]:
# сохранение модели
model.save_model('artifacts/catboost/embeddings_and_heuristics_model.cbm')